# Sensitivity to ISO

In [1]:
using ModelingToolkit
using OrdinaryDiffEq
using DiffEqCallbacks
using Plots
using LsqFit
using CaMKIIModel
using CaMKIIModel: μM, hil, Hz, hilr
Plots.default(lw=1.5, fmt=:png)

## Setup b1AR system

In [ ]:
@parameters ATP = 5000μM ISO = 0μM
sys = get_bar_sys(ATP, ISO; simplify=true)

In [ ]:
tend = 10000.0
prob = ODEProblem(sys, [], tend)
alg = Rodas5()
callback = TerminateSteadyState()

# Log scale for ISO concentration
iso = logrange(1e-6μM, 10μM, 1001)

In [ ]:
prob_func = (prob, i, repeat) -> begin
    remake(prob, p=[ISO => iso[i]])
end
trajectories = length(iso)
sol = solve(prob, alg; callback) ## warmup
sim = solve(EnsembleProblem(prob; prob_func, safetycopy=false), alg; trajectories, callback)

## time series

It took about 1000 seconds to reach equilibrium.

In [ ]:
plot(sol, idxs=sys.PKACI / sys.RItot)

In [ ]:
"""Extract values from ensemble simulations by a symbol"""
extract(sim, k) = map(s -> s[k][end], sim)

In [ ]:
xopts = (xlims=(iso[begin] * 1000, iso[end] * 1000), minorgrid=true, xscale=:log10, xlabel="ISO (μM)",)

plot(iso .* 1000, extract(sim, sys.cAMP); lab="cAMP", ylabel="Conc. (mM)", legend=:topleft, xopts...)

In [ ]:
plot(iso .* 1000, extract(sim, sys.PKACI / sys.RItot); lab="PKACI", ylabel="Activation fraction")
plot!(iso .* 1000, extract(sim, sys.PKACII / sys.RIItot), lab="PKACII", legend=:topleft; xopts...)

## Least-square fitting of PKACI activity

In [ ]:
@. model(x, p) = p[1] * x / (x + p[2]) + p[3]
xdata = iso
ydata = extract(sim, sys.PKACI / sys.RItot)
p0 = [0.3, 1E-5, 0.08]
lb = [0.0, 0.0, 0.0]
pkac1_fit = curve_fit(model, xdata, ydata, p0; lower=lb)

In [ ]:
pkac1_coef = coef(pkac1_fit)

In [ ]:
println("PKACI")
println("Basal activity: ", pkac1_coef[3])
println("Maximal activity: ", pkac1_coef[1] + pkac1_coef[3])
println("Michaelis constant: ", pkac1_coef[2] * 1000, " μM")

In [ ]:
ypred = model.(xdata, Ref(pkac1_coef))
p1 = plot(xdata .* 1000, [ydata ypred], lab=["Full model" "Fitted"], line=[:dash :dot], title="PKACI", legend=:topleft; xopts...)
p2 = plot(xdata .* 1000, (ypred .- ydata) ./ ydata .* 100; title="PKACI error (%)", lab=false, xopts...)
plot(p1, p2, layout=(2, 1), size=(600, 600))

## Least-square fitting of PKACII activity

In [ ]:
xdata = iso
ydata = extract(sim, sys.PKACII / sys.RIItot)
p0 = [0.4, 0.01μM, 0.2]
lb = [0.0, 0.0, 0.0]
pkac2_fit = curve_fit(model, xdata, ydata, p0; lower=lb)

In [ ]:
pkac2_coef = coef(pkac2_fit)

In [ ]:
println("PKACII")
println("Basal activity: ", pkac2_coef[3])
println("Maximal activity: ", pkac2_coef[1] + pkac2_coef[3])
println("Michaelis constant: ", pkac2_coef[2] * 1000, " μM")

In [ ]:
ypred = model.(xdata, Ref(pkac2_coef))
p1 = plot(xdata .* 1000, [ydata ypred], lab=["Full model" "Fitted"], line=[:dash :dot], title="PKACII", legend=:topleft; xopts...)
p2 = plot(xdata .* 1000, (ypred .- ydata) ./ ydata .* 100; title="PKACII error (%)", lab=false, xopts...)
plot(p1, p2, layout=(2, 1), size=(600, 600))

## Least-square fitting of PP1 activity

In [ ]:
@. model_pp1(x, p) = p[1] * p[2] / (x + p[2]) + p[3]
xdata = iso
ydata = extract(sim, sys.PP1 / sys.PP1tot)
p0 = [0.1, 8e-6, 0.8]
lb = [0.0, 0.0, 0.0]
pp1_fit = curve_fit(model_pp1, xdata, ydata, p0; lower=lb)

In [ ]:
pp1_coef = coef(pp1_fit)

In [ ]:
println("PP1")
println("Basal activity: ", pp1_coef[1] + pp1_coef[3])
println("Minimal activity: ", pp1_coef[3])
println("Repressive Michaelis constant: ", pp1_coef[2] * 1000, " μM")

In [ ]:
ypred = model_pp1.(xdata, Ref(pp1_coef))
p1 = plot(xdata .* 1000, [ydata ypred], lab=["Full model" "Fitted"], line=[:dash :dot], title="PP1", legend=:topleft; xopts...)
p2 = plot(xdata .* 1000, (ypred .- ydata) ./ ydata .* 100; title="PP1 error (%)", lab=false, xopts...)
plot(p1, p2, layout=(2, 1), size=(600, 600))

## Least-square fitting of PLBp

In [ ]:
xdata = iso
ydata = extract(sim, sys.PLBp / sys.PLBtot)

plot(xdata .* 1000, ydata, title="PLBp activity", lab=false; xopts...)

First try: Hill function

In [ ]:
@. model_plb(x, p) = p[1] * hil(x, p[2], p[3]) + p[4]
p0 = [0.8, 1e-5, 1.0, 0.1]
lb = [0.5, 1e-9, 1.0, 0.0]
fit = curve_fit(model_plb, xdata, ydata, p0; lower=lb, autodiff=:forwarddiff)

In [ ]:
pestim = coef(fit)

In [ ]:
ypred = model_plb.(xdata, Ref(pestim))
p1 = plot(xdata .* 1000, [ydata ypred], lab=["Full model" "Fitted"], line=[:dash :dot], title="PLBp", legend=:topleft; xopts...)
p2 = plot(xdata .* 1000, (ypred .- ydata) ./ ydata .* 100; title="PLBp error (%)", lab=false, xopts...)
plot(p1, p2, layout=(2, 1), size=(600, 600))

Second try: analytic PLB from the quadratic equation

In [ ]:
function plbp_analytic(iso)
    PLBtot = 106μM
    PKACItot = 1.18μM
    PP1tot = 0.89μM
    k_PKA_PLB = 54Hz
    Km_PKA_PLB = 21μM
    k_PP1_PLB = 8.5Hz
    Km_PP1_PLB = 7.0μM
    PKACI_basal = 0.0831  ## basal activity
    PKACI_activated = 0.25603
    PKACI_KM = 0.0144μM
    PP1_basal = 0.82365
    PP1_activated = 0.1025
    PP1_KI = 0.008465μM

    ## Solve for Vf * x / (x + k1) = Vr * (1 - x) / (1 - x + k2)
    PKACI = PKACItot * (PKACI_basal + PKACI_activated * hil(iso, PKACI_KM))
    PP1 = PP1tot * (PP1_basal + PP1_activated * hilr(iso, PP1_KI))

    Vf = k_PKA_PLB * PKACI
    k1 = Km_PKA_PLB / PLBtot
    Vr = k_PP1_PLB * PP1
    k2 = Km_PP1_PLB / PLBtot
    A = 1 - (Vf / Vr)
    B = (Vf / Vr) * (1 + k2) + (k1 - 1)
    C = -k1
    plb = ifelse(A ≈ 0.0, k1 / (k1 + k2), (-B + sqrt(B^2 - 4 * A * C)) / 2A)
    plbp = 1 - plb
end

In [ ]:
ypred = plbp_analytic.(xdata)
p1 = plot(xdata .* 1000, [ydata ypred], lab=["Full model" "Quadratic"], line=[:dash :dot], title="PLBp", legend=:topleft; xopts...)
p2 = plot(xdata .* 1000, (ypred .- ydata) ./ ydata .* 100; title="PLBp error (%)", lab=false, xopts...)
plot(p1, p2, layout=(2, 1), size=(600, 600))